<a href="https://colab.research.google.com/github/Chiamakac/TRAININGS/blob/main/Spacy_IgboNER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Developing IgboNER tagger with Spacy

In [ ]:
!wget https://raw.githubusercontent.com/masakhane-io/masakhane-ner/main/data/ibo/dev.txt

--2023-09-26 17:35:26--  https://raw.githubusercontent.com/masakhane-io/masakhane-ner/main/data/ibo/dev.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53684 (52K) [text/plain]
Saving to: ‘dev.txt’

dev.txt             100%[===================>]  52.43K  --.-KB/s    in 0.01s   

2023-09-26 17:35:26 (3.76 MB/s) - ‘dev.txt’ saved [53684/53684]



In [ ]:
# Read the lines from the 'dev.txt' file, strip leading/trailing spaces and split by spaces
# This creates a list of lists where each sublist contains words from a line
lines = [line.strip().split() for line in open('dev.txt').readlines()]

# Extract the first word from each sublist and join them into a single string with spaces in between
# This creates a string containing the first word of each non-empty line
text = " ".join([l[0] for l in lines if l])


In [ ]:
def join_spans(tag_list):
  i=0
  tag_spans = []
  while i < len(tag_list):
    token, tag = tag_list[i]
    while i+1< len(tag_list) and tag_list[i+1][1].startswith('I'):
      token = token + ' ' + tag_list[i+1][0]
      i += 1
    tag_spans.append((token,tag))
    i += 1
  return [(t, s[2:]) for t, s in tag_spans]
ner_taglist = join_spans([l for l in lines if l and l[1]!='O'])

In [ ]:
locs =  [l[0] for l in ner_taglist if l[1].endswith('LOC')]
dates =  [l[0] for l in ner_taglist if l[1].endswith('DATE')]
orgs =  [l[0] for l in ner_taglist if l[1].endswith('ORG')]
pers =  [l[0] for l in ner_taglist if l[1].endswith('PER') if l[0]!='-']

In [ ]:
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"]) # we will change the English model later...

# add spacy's EntityRuler
ruler = nlp.add_pipe('entity_ruler')#, before='ner')

# add entity patterns
patterns  = [{'label':'LOC', 'pattern':loc} for loc in set(locs)]
patterns += [{'label':'DATE', 'pattern':date} for date in set(dates)]
patterns += [{'label':'ORG', 'pattern':orgs} for orgs in set(orgs)]
patterns += [{'label':'PER', 'pattern':pers} for pers in set(pers)]
ruler.add_patterns(patterns)

In [ ]:
sentences = """Buhari nọ n'isi gọọmentị mpụ na nrụrụaka - Saraki Ọ pụtara na enweghi ihe e mere mekaa ndị Igbo ?
ASUU strike : Kedụ mgbe ọ ga - ebi ?
Sineti sị gọọmenti kpọchita Onnoghen Ndị Sịnetị a bụ nke onyeisi ha bụ Bukola Saraki bụ nke agba asatọ n'ime ọchịchị onye kwuo uche ya.
Ndị Sịnetị a bụ nke onyeisi ha bụ Bukola Saraki bụ nke agba asatọ n'ime ọchịchị onye - kwuo - uche - ya
N'aka ọzọ , ụlọomeiwu Sineti akpọpụla gọọmenti etiti ụlọikpe ka ha were ọsọ were ije kpọchite Onnoghen bụ onye bụbụ onyeisi ndị okaikpe na Naịjirịa .""".splitlines()

In [ ]:
doc = nlp(sentences[4])
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
displacy.render(doc, style="ent",  jupyter=True)

Onnoghen 94 102 PER
Naịjirịa 139 147 LOC


/usr/local/lib/python3.10/dist-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [ ]:
print(text)

N'oge mbụ , ụmụnwaanyị na - eke isi dị iche iche , kama na ọ na - abụkarị isi ojii Mana ugbua , ha na - anwale ụdị agwa sọrọ ha , site na isi na - acha ọbara ọbara ruo na nke na - acha edo edo , ha achọghị ịma . Ha na - eme oriri na nkwari ka o si dị ha mma Oge agaala mgbe nwaanyị ga - anọ n'ụlọ , nwoke apụo inwete onwe ya naanị ya . Ugbua , ụmụnwanyị na - akpọrọ ndị enyi ha puo ebe ọ na - akpọtụ , na - enwe anụrị . Ọ naghị ama akwa , ọ na - eyi chebiriọdụ anọ n'ulọ Anyi ahapụla ima akwa na '19 kporogodom' Oge ima akwa n'obi nọrọ n'ụlọ agafeela . Ugbua ụmụnwaanyị na - eyi chebiriọdụ dị iche iche . I yitekwaraala ihe ndị Bekee kpọrọ 'bum short' , anyi amara n'iso na ihe a na - eme . Ọ na - ewepụta oge eji eme soshal media Teknụzụ emeela na onye ọbụla nọ n'ụwa nwere ike ijikọ Nwaanyị agba ọhụrụ ga - akwado , n'eti ka ọnwa , see onwe ya foto , tinye ya n'akara soshal midia ya . Ọ na - ete ntecha etu o siri masị ya Ụmụnwaanyị dịbu na mbu ete ntecha , kama ugbua , e nwezịrị agwa dị iche ich